In [1]:
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

In [4]:
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

flights_df = flights_df.fillna( { 'DEPARTURE_DELAY':0 } )

from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

flights_df = flights_df.drop('ARRIVAL_DELAY')


In [5]:
flights_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = false)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- ARRIVAL_DELAY: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [6]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in flights_df.columns]).show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|Distance_Bucket|Flight_Distance|Airline_Numeric|Airline_OHE|OA_Numeric|Origin_Airport_OHE|DA_Numeric|Destination_Airport_OHE|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+

In [7]:
from pyspark.sql import functions as fn
flights_df.groupBy('CANCELLED').agg(fn.count('*')).show()

+---------+--------+
CANCELLED|count(1)|
+---------+--------+
 1| 89884|
 0| 5729195|
+---------+--------+

In [8]:
training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

In [9]:
major_df = training_df.filter(col("CANCELLED") == 0)
minor_df = training_df.filter(col("CANCELLED") == 1)

In [10]:
ratio = major_df.count()/minor_df.count()
print(ratio)

63.658002938670684

In [11]:
sampled_majority_df = major_df.sample(False, 3/ratio, seed=5)
combined_df_2 = sampled_majority_df.unionAll(minor_df)

In [12]:
combined_df_2.groupBy('CANCELLED').agg(fn.count('*')).show()

+---------+--------+
CANCELLED|count(1)|
+---------+--------+
 1| 80989|
 0| 81086|
+---------+--------+

In [13]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL"], outputCol="features")

In [14]:
inputColumns = ["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL"]

In [15]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidatorModel
bce = BinaryClassificationEvaluator(labelCol='CANCELLED', metricName='areaUnderROC')
from pyspark.ml import evaluation

In [16]:
lr = LogisticRegression(featuresCol='features', labelCol='CANCELLED')
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)

In [17]:
predictions_train = lr_transform
y_true = predictions_train.select(['CANCELLED']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.98 1.00 0.99 573597
 1 0.14 0.01 0.02 8895

 micro avg 0.98 0.98 0.98 582492
 macro avg 0.56 0.50 0.51 582492
weighted avg 0.97 0.98 0.98 582492

Out[44]: array([[573030, 567],
 [ 8800, 95]])

In [18]:
plt.figure(figsize=(10,6))
plt.plot([0, 1], [0, 1], 'r--')
plt.scatter(lr_model.stages[-1].summary.roc.select('FPR').collect(),
            lr_model.stages[-1].summary.roc.select('TPR').collect())
plt.title('ROC Scatter Plot Flight Cancellation : TPR/FPR')
plt.xlabel('FPR')
plt.ylabel('TPR')
display()

In [19]:
lr_bce = BinaryClassificationEvaluator(labelCol='CANCELLED', metricName='areaUnderROC')
lr_bce.evaluate(lr_transform)

Out[60]: 0.759853395512296

In [20]:
lr_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="accuracy")
lr_evaluator_accuracy.evaluate(lr_transform)

Out[61]: 0.9287612533734368

In [21]:
rf = RandomForestClassifier(featuresCol='features', labelCol='CANCELLED')
rf_pipeline = Pipeline(stages=[va, rf])
rf_model = rf_pipeline.fit(combined_df_2)
rf_transform = rf_model.transform(testing_df)

In [23]:
predictions_train = rf_transform
y_true = predictions_train.select(['CANCELLED']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
 precision recall f1-score support

 0 0.98 1.00 0.99 573597
 1 0.00 0.00 0.00 8895

 micro avg 0.98 0.98 0.98 582492
 macro avg 0.49 0.50 0.50 582492
weighted avg 0.97 0.98 0.98 582492

Out[47]: array([[573597, 0],
 [ 8895, 0]])

In [24]:
rf_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="accuracy")
rf_evaluator_accuracy.evaluate(rf_transform)

Out[50]: 0.9847294040089821

In [25]:
rf_bce = BinaryClassificationEvaluator(labelCol='CANCELLED', metricName='areaUnderROC')
rf_bce.evaluate(rf_transform)

Out[51]: 0.6913142546391783

In [26]:
randomForest_model.featureImportances.toArray()

Out[25]: array([0.09693428, 0.02103434, 0.04248902, ..., 0.01811722, 0.00213839,
 0.23430568])

In [27]:
randomForest_model = rf_model.stages[-1]
featureImportance_RF = pd.DataFrame(list(zip(inputColumns, randomForest_model.featureImportances.toArray())),
             columns = ['column', 'weight']).sort_values('weight', ascending=False)

In [28]:
ax1 = sns.barplot(x='weight', y='column', data=featureImportance_RF)
ax1.set(xlabel='Importance', ylabel='Features')
display()

In [29]:
gbt = GBTClassifier(featuresCol='features', labelCol='CANCELLED')
gbt_pipeline = Pipeline(stages=[va, gbt])
gbt_model = gbt_pipeline.fit(combined_df_2)
gbt_transform = gbt_model.transform(testing_df)

In [30]:
gbt_bce = BinaryClassificationEvaluator(labelCol='CANCELLED',metricName='areaUnderROC')
gbt_bce.evaluate(gbt_transform)

Out[56]: 0.8051252833632779

In [31]:
plt.figure(figsize=(5,5))
plt.plot([0, 1], [0, 1], 'r--')
plt.scatter(gbt_model.stages[-1].summary.roc.select('FPR').collect(),
            gbt_model.stages[-1].summary.roc.select('TPR').collect())
plt.title('ROC Scatter Plot : TPR/FPR')
plt.xlabel('FPR')
plt.ylabel('TPR')
display()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2795822308928804> in <module> 
 1 plt . figure ( figsize = ( 5 , 5 ) ) 
 2 plt . plot ( [ 0 , 1 ] , [ 0 , 1 ] , 'r--' ) 
 ----> 3 plt.scatter(gbt_model.stages[-1].summary.roc.select('FPR').collect(),
 4 gbt_model.stages[-1].summary.roc.select('TPR').collect())
 5 plt . title ( 'ROC Scatter Plot : TPR/FPR' ) 

 AttributeError : 'GBTClassificationModel' object has no attribute 'summary'

In [32]:
boost_model = gbt_model.stages[-1]
gbt_df = pd.DataFrame(list(zip(inputColumns, boost_model.featureImportances.toArray())),
             columns = ['column', 'weight']).sort_values('weight', ascending=False)

In [33]:
ax1 = sns.barplot(x='weight', y='column', data=gbt_df)
ax1.set(xlabel='Importance', ylabel='Features')
display()

In [34]:
predictions_train = gbt_transform
y_true = predictions_train.select(['CANCELLED']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.99 0.96 0.97 573597
 1 0.10 0.33 0.16 8895

 micro avg 0.95 0.95 0.95 582492
 macro avg 0.55 0.64 0.57 582492
weighted avg 0.98 0.95 0.96 582492

Out[55]: array([[548400, 25197],
 [ 5948, 2947]])

In [35]:
gbt_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="f1")
gbt_evaluator_f1.evaluate(gbt_transform)

In [36]:
gbt_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="accuracy")
gbt_evaluator_accuracy.evaluate(gbt_transform)

Out[57]: 0.9465314545092465

In [37]:
gbt_evaluator_precision = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="weightedPrecision")
gbt_evaluator_precision.evaluate(gbt_transform)

In [38]:
gbt_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="weightedRecall")
gbt_evaluator_recall.evaluate(gbt_transform)